In [1]:
##from pprint import pprint
##import time
##import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

headers={
    'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.49",
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive', 
    'Upgrade-Insecure-Requests': '1',
    'Cach-Control': 'max-age=0'
}
base_url= "https://www.coingecko.com/"

tables = []


for i in range(1,5):
    print('Processing page {0}'.format(i))
    params= {
        'page': i
    }
    response = requests.get(base_url,headers=headers,params=params)
    soup = BeautifulSoup(response.content,'html.parser')
    tables.append(pd.read_html(str(soup))[0])
    
master_table=pd.concat(tables)
master_table =master_table.loc[:, master_table.columns[1:-1]]
master_table.to_csv('Crypto Data Table.csv', index =False)

Processing page 1
Processing page 2
Processing page 3
Processing page 4


In [2]:
pd.read_html(str(soup))[0]

,Unnamed: 0,#,Coin,Unnamed: 3,Price,1h,24h,7d,24h Volume,Mkt Cap,FDV,Last 7 Days
0,NaN,301,Poollotto.finance PLT PLT,NaN,$7.31,2.9%,-5.5%,5.5%,"$790,919","$77,908,973",?,NaN
1,NaN,302,Nerve Finance NRV NRV,NaN,$0.006609654627,-0.0%,-9.0%,-10.3%,"$27,260.36","$77,747,624","$156,256,967",NaN
2,NaN,303,CUBE ITAMCUBE ITAMCUBE,NaN,$0.110476,0.8%,-2.9%,-7.0%,"$75,313.21","$77,470,658","$1,104,953,599",NaN
3,NaN,304,flexUSD FLEXUSD FLEXUSD,NaN,$0.392108,14.0%,-3.1%,12.1%,"$5,129.27","$77,432,250",?,NaN
4,NaN,306,Creditcoin CTC CTC,NaN,$0.370463,-0.4%,0.3%,-2.6%,"$4,869,072","$77,105,719","$744,068,777",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,395,Blox CDT CDT,NaN,$0.079646634455,5.7%,-21.5%,-39.6%,"$93,848.66","$51,100,821",?,NaN
96,NaN,397,Phoenix Global [OLD] PHB PHB,NaN,$0.915194,0.0%,0.0%,0.0%,$0.000000000000,"$50,271,405","$50,350,229",NaN
97,NaN,398,Koda Cryptocurrency KODA KODA,NaN,$0.000101700120,0.1%,-5.8%,2.4%,$898.34,"$50,208,210","$41,747,070",NaN
98,NaN,401,DUSK Network DUSK DUSK,NaN,$0.122090,1.2%,-3.5%,-8.0%,"$8,289,763","$50,058,907","$122,071,511",NaN


In [3]:
#Clean HTML

def clean_page(html, pretty_print=False):
    """
    >>> junk = "some random HTML<P> for you to try to parse</p>"
    >>> clean_page(junk)
    '<div><p>some random HTML</p><p> for you to try to parse</p></div>'
    >>> print clean_page(junk, pretty_print=True)
    <div>
    <p>some random HTML</p>
    <p> for you to try to parse</p>
    </div>
    """
    from lxml.html import fromstring
    from lxml.html import tostring
    return tostring(fromstring(html), pretty_print=pretty_print)

In [4]:
#Converting yes/no to Boolean values

def to_bool(yes_no, none_to_false=True):
    """
    >>> to_bool('')
    False
    >>> to_bool(None):
    False
    >>> to_bool('y')
    True
    >>> to_bool('yip')
    True
    >>> to_bool('Yes')
    True
    >>> to_bool('nuh')
    False
    """
    yes_no = yes_no.strip().lower()
    if not yes_no.strip() and none_to_false:
        return False
    if yes_no.startswith('y'):
        return True
    elif yes_no.startswith('n'):
        return False

In [5]:
#Converting numbers to the correct type

def to_int(number, european=False):
    """
    >>> to_int('32')
    32
    >>> to_int('3,998')
    3998
    >>> to_int('3.998', european=True)
    3998
    """
    if european:
        number = number.replace('.', '')
    else:
        number = number.replace(',', '')
    return int(number)

def to_float(number, european=False):
    """
    >>> to_float(u'42.1')
    42.1
    >>> to_float(u'32,1', european=True)
    32.1
    >>> to_float('3,132.87')
    3132.87
    >>> to_float('3.132,87')
    3132.87
    >>> to_float('(54.12)')
    -54.12

    Warning
    -------

    Incorrectly declaring `european` leads to troublesome results:

    >>> to_float('54.2', european=True)
    542
    """
    import string
    if european:
        table = string.maketrans(',.','.,')
        number = string.translate(number, table)
    number = number.replace(',', '')
    if number.startswith('(') and number.endswith(')'):
        number = '-' + number[1:-1]
    return float(number)

In [6]:
pd.set_option('display.float_format',lambda x:'%5f'% x)

In [7]:
>>> u'\n\tTitle'.strip()
u'Title'

'Title'

In [8]:
def date_to_iso(datestring):
    """
    Takes a string of a human-readable date and
    returns a machine-readable date string.


    >>> date_to_iso('20 July 2002')
    '2002-07-20 00:00:00'
    >>> date_to_iso('June 3 2009 at 4am')
    '2009-06-03 04:00:00'
    """
    from dateutil import parser
    from datetime import datetime
    default = datetime(year=1, month=1, day=1)
    return str(parser.parse(datestring, default=default))

In [9]:
master_table

,#,Coin,Unnamed: 3,Price,1h,24h,7d,24h Volume,Mkt Cap,FDV
0,1,Bitcoin BTC BTC,NaN,"$21,180.55",0.4%,-3.2%,-5.4%,"$46,721,161,879","$404,946,078,918","$445,147,101,698"
1,2,Ethereum ETH ETH,NaN,"$1,430.31",0.5%,-5.6%,-8.9%,"$19,802,843,324","$171,591,101,666",?
2,3,Tether USDT USDT,NaN,$1.00,0.1%,0.1%,0.0%,"$56,621,867,790","$65,990,115,153",?
3,4,USD Coin USDC USDC,NaN,$1.00,0.0%,0.1%,0.1%,"$5,583,163,898","$55,090,691,590",?
4,5,BNB BNB BNB,NaN,$244.07,0.1%,-4.4%,-7.6%,"$824,278,535","$39,834,672,958","$40,283,525,275"
...,...,...,...,...,...,...,...,...,...,...
95,395,Blox CDT CDT,NaN,$0.079646634455,5.7%,-21.5%,-39.6%,"$93,848.66","$51,100,821",?
96,397,Phoenix Global [OLD] PHB PHB,NaN,$0.915194,0.0%,0.0%,0.0%,$0.000000000000,"$50,271,405","$50,350,229"
97,398,Koda Cryptocurrency KODA KODA,NaN,$0.000101700120,0.1%,-5.8%,2.4%,$898.34,"$50,208,210","$41,747,070"
98,401,DUSK Network DUSK DUSK,NaN,$0.122090,1.2%,-3.5%,-8.0%,"$8,289,763","$50,058,907","$122,071,511"


In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
master_table.describe()

,#,Unnamed: 3
count,400.000000,0.000000
mean,200.510000,NaN
std,115.629388,NaN
min,1.000000,NaN
25%,100.750000,NaN
50%,200.500000,NaN
75%,300.250000,NaN
max,401.000000,NaN


In [12]:
master_table.head()

,#,Coin,Unnamed: 3,Price,1h,24h,7d,24h Volume,Mkt Cap,FDV
0,1,Bitcoin BTC BTC,NaN,"$21,180.55",0.4%,-3.2%,-5.4%,"$46,721,161,879","$404,946,078,918","$445,147,101,698"
1,2,Ethereum ETH ETH,NaN,"$1,430.31",0.5%,-5.6%,-8.9%,"$19,802,843,324","$171,591,101,666",?
2,3,Tether USDT USDT,NaN,$1.00,0.1%,0.1%,0.0%,"$56,621,867,790","$65,990,115,153",?
3,4,USD Coin USDC USDC,NaN,$1.00,0.0%,0.1%,0.1%,"$5,583,163,898","$55,090,691,590",?
4,5,BNB BNB BNB,NaN,$244.07,0.1%,-4.4%,-7.6%,"$824,278,535","$39,834,672,958","$40,283,525,275"


In [13]:
master_table.duplicated()

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Length: 400, dtype: bool

In [14]:
print(master_table.isnull().sum())


#               0
Coin            0
Unnamed: 3    400
Price           0
1h              0
24h             0
7d              0
24h Volume      0
Mkt Cap         0
FDV             0
dtype: int64


In [15]:
master_table[master_table.Coin.str.contains("Bitcoin")]

,#,Coin,Unnamed: 3,Price,1h,24h,7d,24h Volume,Mkt Cap,FDV
0,1,Bitcoin BTC BTC,NaN,"$21,180.55",0.4%,-3.2%,-5.4%,"$46,721,161,879","$404,946,078,918","$445,147,101,698"
17,18,Wrapped Bitcoin WBTC WBTC,NaN,"$21,164.86",0.3%,-3.4%,-5.4%,"$334,800,675","$5,004,215,725","$5,004,215,725"
30,31,Bitcoin Cash BCH BCH,NaN,$116.48,-0.3%,-8.6%,-4.3%,"$267,903,788","$2,228,501,936","$2,446,656,311"
52,53,Bitcoin SV BSV BSV,NaN,$53.53,0.2%,-5.6%,-6.7%,"$26,854,962","$1,024,233,766",?
7,108,Bitcoin Gold BTG BTG,NaN,$20.65,0.6%,-2.5%,5.6%,"$31,487,248","$361,729,272",?


In [16]:
# delete a column
 
del master_table['Unnamed: 3']
master_table

,#,Coin,Price,1h,24h,7d,24h Volume,Mkt Cap,FDV
0,1,Bitcoin BTC BTC,"$21,180.55",0.4%,-3.2%,-5.4%,"$46,721,161,879","$404,946,078,918","$445,147,101,698"
1,2,Ethereum ETH ETH,"$1,430.31",0.5%,-5.6%,-8.9%,"$19,802,843,324","$171,591,101,666",?
2,3,Tether USDT USDT,$1.00,0.1%,0.1%,0.0%,"$56,621,867,790","$65,990,115,153",?
3,4,USD Coin USDC USDC,$1.00,0.0%,0.1%,0.1%,"$5,583,163,898","$55,090,691,590",?
4,5,BNB BNB BNB,$244.07,0.1%,-4.4%,-7.6%,"$824,278,535","$39,834,672,958","$40,283,525,275"
...,...,...,...,...,...,...,...,...,...
95,395,Blox CDT CDT,$0.079646634455,5.7%,-21.5%,-39.6%,"$93,848.66","$51,100,821",?
96,397,Phoenix Global [OLD] PHB PHB,$0.915194,0.0%,0.0%,0.0%,$0.000000000000,"$50,271,405","$50,350,229"
97,398,Koda Cryptocurrency KODA KODA,$0.000101700120,0.1%,-5.8%,2.4%,$898.34,"$50,208,210","$41,747,070"
98,401,DUSK Network DUSK DUSK,$0.122090,1.2%,-3.5%,-8.0%,"$8,289,763","$50,058,907","$122,071,511"


In [17]:
master_table.describe()

,#
count,400.000000
mean,200.510000
std,115.629388
min,1.000000
25%,100.750000
50%,200.500000
75%,300.250000
max,401.000000


In [18]:
master_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 99
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           400 non-null    int64 
 1   Coin        400 non-null    object
 2   Price       400 non-null    object
 3   1h          400 non-null    object
 4   24h         400 non-null    object
 5   7d          400 non-null    object
 6   24h Volume  400 non-null    object
 7   Mkt Cap     400 non-null    object
 8   FDV         400 non-null    object
dtypes: int64(1), object(8)
memory usage: 31.2+ KB
